# 🐕 Обучение классификатора пород собак

**Sprint 3: Breed Classification**

Этот notebook обучает EfficientNet-B4 на Stanford Dogs Dataset.

## Требования
- Google Colab с GPU (Runtime → Change runtime type → GPU)
- Google Drive для сохранения весов

## Результат
- Обученная модель `breed_best.pt`
- Top-5 accuracy > 80%

---
## 0. Проверка GPU

In [ ]:
import torch
import os

# Определяем папку проекта (будет использоваться позже)
PROJECT_DIR = '/content/drive/MyDrive/DogFACS'

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU доступен: {gpu_name}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ GPU не доступен!")
    print("   Перейди в Runtime → Change runtime type → GPU")

---
## 1. Установка зависимостей

In [ ]:
!pip install -q timm kaggle tqdm

---
## 2. Подключение Google Drive

Для сохранения обученной модели.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Создаём папку для проекта
import os
PROJECT_DIR = '/content/drive/MyDrive/DogFACS'
os.makedirs(PROJECT_DIR, exist_ok=True)
print(f"✅ Папка проекта: {PROJECT_DIR}")

---
## 3. Скачивание Stanford Dogs Dataset

### Настройка Kaggle API

**Вариант A (рекомендуется):** Введи данные прямо в код ниже:
1. Зайди на https://www.kaggle.com/settings
2. Найди секцию "API" и скопируй:
   - Username (твой логин на Kaggle)
   - API Key (нажми "Create New Token" если нет)
3. Вставь в ячейку ниже

**Вариант B:** Загрузи файл `kaggle.json` если он у тебя есть

In [ ]:
# Настройка Kaggle API
# Вариант 1: Введи username и API key вручную
KAGGLE_USERNAME = ""  # ← Введи свой username
KAGGLE_KEY = ""       # ← Введи свой API key

# Если заполнил выше - используем их
if KAGGLE_USERNAME and KAGGLE_KEY:
    import json
    kaggle_config = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}
    
    !mkdir -p ~/.kaggle
    with open("/root/.kaggle/kaggle.json", "w") as f:
        json.dump(kaggle_config, f)
    !chmod 600 ~/.kaggle/kaggle.json
    print("✅ Kaggle настроен через ввод ключа")

# Вариант 2: Загрузи файл kaggle.json (если есть)
else:
    from google.colab import files
    print("Username и Key не заполнены.")
    print("Загрузи файл kaggle.json или заполни KAGGLE_USERNAME и KAGGLE_KEY выше")
    try:
        uploaded = files.upload()
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        print("✅ Kaggle настроен через файл")
    except:
        print("❌ Файл не загружен. Заполни KAGGLE_USERNAME и KAGGLE_KEY")

In [ ]:
# Скачиваем dataset
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset
!unzip -q stanford-dogs-dataset.zip -d /content/stanford_dogs
!rm stanford-dogs-dataset.zip

print("✅ Stanford Dogs Dataset скачан")
!ls /content/stanford_dogs/

### Вариант B: Прямая ссылка (если нет Kaggle)

In [ ]:
# Раскомментируй если используешь вариант B
# !wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
# !mkdir -p /content/stanford_dogs/images
# !tar -xf images.tar -C /content/stanford_dogs/
# !rm images.tar

---
## 4. Подготовка данных

In [ ]:
import json
import shutil
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm

# Пути
RAW_DIR = Path("/content/stanford_dogs")
OUTPUT_DIR = Path("/content/breed_training")

# Найдём папку с изображениями
if (RAW_DIR / "images" / "Images").exists():
    IMAGES_DIR = RAW_DIR / "images" / "Images"
elif (RAW_DIR / "Images").exists():
    IMAGES_DIR = RAW_DIR / "Images"
else:
    # Поиск
    for p in RAW_DIR.rglob("n02*"):
        IMAGES_DIR = p.parent
        break

print(f"Папка с изображениями: {IMAGES_DIR}")
print(f"Найдено папок пород: {len(list(IMAGES_DIR.iterdir()))}")

In [ ]:
def parse_breed_name(folder_name: str) -> str:
    """Извлекает название породы из имени папки."""
    if "-" in folder_name:
        breed = folder_name.split("-", 1)[1]
    elif "_" in folder_name:
        parts = folder_name.split("_")
        if parts[0].startswith("n"):
            breed = "_".join(parts[1:])
        else:
            breed = folder_name
    else:
        breed = folder_name
    return breed.replace("_", " ")

# Собираем все породы
breeds_data = {}
for breed_folder in sorted(IMAGES_DIR.iterdir()):
    if not breed_folder.is_dir():
        continue
    breed_name = parse_breed_name(breed_folder.name)
    images = list(breed_folder.glob("*.jpg")) + list(breed_folder.glob("*.JPEG"))
    if images:
        breeds_data[breed_name] = images

print(f"\nНайдено {len(breeds_data)} пород")
total_images = sum(len(imgs) for imgs in breeds_data.values())
print(f"Всего изображений: {total_images}")

# Показать несколько примеров
print("\nПримеры пород:")
for i, (breed, imgs) in enumerate(list(breeds_data.items())[:5]):
    print(f"  {breed}: {len(imgs)} изображений")

In [ ]:
# Создаём маппинг ID → название породы
sorted_breeds = sorted(breeds_data.keys())
breed_to_id = {breed: i for i, breed in enumerate(sorted_breeds)}
id_to_breed = {i: breed for i, breed in enumerate(sorted_breeds)}

print(f"Количество классов: {len(id_to_breed)}")

# Сохраняем маппинг
breeds_json = {str(k): v for k, v in id_to_breed.items()}
with open("/content/breeds.json", "w") as f:
    json.dump(breeds_json, f, indent=2)
print("✅ breeds.json сохранён в /content/")

# Копируем в Google Drive (если подключён)
try:
    if os.path.exists("/content/drive/MyDrive"):
        os.makedirs(PROJECT_DIR, exist_ok=True)
        shutil.copy("/content/breeds.json", f"{PROJECT_DIR}/breeds.json")
        print(f"✅ Скопировано в Google Drive: {PROJECT_DIR}")
    else:
        print("⚠️ Google Drive не подключён - пропускаем копирование")
except Exception as e:
    print(f"⚠️ Не удалось скопировать в Drive: {e}")

In [ ]:
# Разделение на train/val/test (80/10/10)
np.random.seed(42)

splits = {"train": {}, "val": {}, "test": {}}

for breed, images in breeds_data.items():
    indices = np.random.permutation(len(images))
    
    train_end = int(len(images) * 0.8)
    val_end = train_end + int(len(images) * 0.1)
    
    splits["train"][breed] = [images[i] for i in indices[:train_end]]
    splits["val"][breed] = [images[i] for i in indices[train_end:val_end]]
    splits["test"][breed] = [images[i] for i in indices[val_end:]]

# Статистика
for split_name, data in splits.items():
    total = sum(len(imgs) for imgs in data.values())
    print(f"{split_name}: {total} изображений")

In [ ]:
# Копируем изображения в формат ImageFolder
print("Копирование изображений...")

for split_name, breeds in splits.items():
    split_dir = OUTPUT_DIR / split_name
    
    for breed_name, images in tqdm(breeds.items(), desc=split_name):
        breed_id = breed_to_id[breed_name]
        safe_name = breed_name.replace(" ", "_").replace("/", "-")
        breed_folder = split_dir / f"{breed_id:03d}_{safe_name}"
        breed_folder.mkdir(parents=True, exist_ok=True)
        
        for i, img_path in enumerate(images):
            dst = breed_folder / f"{i:04d}.jpg"
            shutil.copy2(img_path, dst)

print("\n✅ Данные подготовлены!")
print(f"   {OUTPUT_DIR}")

---
## 5. Настройка обучения

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm

# Конфигурация
CONFIG = {
    "model_name": "efficientnet_b4",
    "num_classes": len(id_to_breed),
    "img_size": 224,
    "batch_size": 32,
    "epochs": 30,
    "lr": 1e-4,
    "weight_decay": 0.01,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

print("Конфигурация обучения:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# Трансформации
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(CONFIG["img_size"], scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    normalize,
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(CONFIG["img_size"]),
    transforms.ToTensor(),
    normalize,
])

print("✅ Трансформации настроены")

In [ ]:
# Датасеты
train_dataset = datasets.ImageFolder(OUTPUT_DIR / "train", transform=train_transform)
val_dataset = datasets.ImageFolder(OUTPUT_DIR / "val", transform=val_transform)
test_dataset = datasets.ImageFolder(OUTPUT_DIR / "test", transform=val_transform)

print(f"Train: {len(train_dataset)} изображений")
print(f"Val: {len(val_dataset)} изображений")
print(f"Test: {len(test_dataset)} изображений")

# DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=CONFIG["batch_size"], 
    shuffle=True, 
    num_workers=2,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=CONFIG["batch_size"], 
    shuffle=False, 
    num_workers=2,
    pin_memory=True,
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=CONFIG["batch_size"], 
    shuffle=False, 
    num_workers=2,
    pin_memory=True,
)

print("✅ DataLoaders созданы")

In [ ]:
# Модель
model = timm.create_model(
    CONFIG["model_name"],
    pretrained=True,
    num_classes=CONFIG["num_classes"],
)
model = model.to(CONFIG["device"])

# Подсчёт параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Модель: {CONFIG['model_name']}")
print(f"   Всего параметров: {total_params:,}")
print(f"   Обучаемых: {trainable_params:,}")

In [ ]:
# Оптимизатор и scheduler
optimizer = AdamW(
    model.parameters(),
    lr=CONFIG["lr"],
    weight_decay=CONFIG["weight_decay"],
)
scheduler = CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])
criterion = nn.CrossEntropyLoss()

print("✅ Оптимизатор: AdamW")
print("✅ Scheduler: CosineAnnealing")
print("✅ Loss: CrossEntropy")

---
## 6. Функции обучения

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Обучение одной эпохи."""
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training", leave=False)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({"loss": f"{loss.item():.4f}", "acc": f"{100.*correct/total:.1f}%"})
    
    return total_loss / total, correct / total


@torch.no_grad()
def validate(model, loader, criterion, device):
    """Валидация с Top-1 и Top-5 accuracy."""
    model.eval()
    total_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Validation", leave=False):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_loss += loss.item() * images.size(0)
        
        # Top-1
        _, predicted = outputs.max(1)
        correct_top1 += predicted.eq(labels).sum().item()
        
        # Top-5
        _, top5_pred = outputs.topk(5, dim=1)
        for i in range(labels.size(0)):
            if labels[i] in top5_pred[i]:
                correct_top5 += 1
        
        total += labels.size(0)
    
    return total_loss / total, correct_top1 / total, correct_top5 / total

---
## 7. Обучение 🚀

In [ ]:
# История обучения
history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_top1": [],
    "val_top5": [],
}

best_top5 = 0.0
best_epoch = 0

print("="*60)
print("🚀 НАЧАЛО ОБУЧЕНИЯ")
print("="*60)
print(f"Эпох: {CONFIG['epochs']}")
print(f"Device: {CONFIG['device']}")
print()

In [ ]:
for epoch in range(CONFIG["epochs"]):
    print(f"\nEpoch {epoch + 1}/{CONFIG['epochs']}")
    print("-" * 40)
    
    # Training
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, CONFIG["device"]
    )
    
    # Validation
    val_loss, val_top1, val_top5 = validate(
        model, val_loader, criterion, CONFIG["device"]
    )
    
    # Scheduler step
    scheduler.step()
    
    # Save history
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_top1"].append(val_top1)
    history["val_top5"].append(val_top5)
    
    # Print metrics
    print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc*100:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Top-1: {val_top1*100:.2f}%, Top-5: {val_top5*100:.2f}%")
    
    # Save best model
    if val_top5 > best_top5:
        best_top5 = val_top5
        best_epoch = epoch + 1
        torch.save(model.state_dict(), "/content/breed_best.pt")
        print(f"  ⭐ Новый лучший результат! Top-5: {best_top5*100:.2f}%")
        
        # Копируем в Google Drive (если подключён)
        try:
            if os.path.exists("/content/drive/MyDrive"):
                os.makedirs(PROJECT_DIR, exist_ok=True)
                shutil.copy("/content/breed_best.pt", f"{PROJECT_DIR}/breed_best.pt")
        except:
            pass

print("\n" + "="*60)
print("✅ ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("="*60)
print(f"Лучший результат: Top-5 = {best_top5*100:.2f}% (epoch {best_epoch})")
print(f"Модель сохранена: /content/breed_best.pt")

---
## 8. Визуализация обучения

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history["train_loss"], label="Train")
axes[0].plot(history["val_loss"], label="Val")
axes[0].set_title("Loss")
axes[0].set_xlabel("Epoch")
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot([x*100 for x in history["train_acc"]], label="Train")
axes[1].plot([x*100 for x in history["val_top1"]], label="Val Top-1")
axes[1].set_title("Top-1 Accuracy")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("%")
axes[1].legend()
axes[1].grid(True)

# Top-5
axes[2].plot([x*100 for x in history["val_top5"]], label="Val Top-5", color="green")
axes[2].axhline(y=80, color="red", linestyle="--", label="Target (80%)")
axes[2].set_title("Top-5 Accuracy")
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("%")
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.savefig("/content/training_history.png", dpi=150)
plt.show()

# Копируем в Google Drive (если подключён)
try:
    if os.path.exists("/content/drive/MyDrive"):
        shutil.copy("/content/training_history.png", f"{PROJECT_DIR}/training_history.png")
        print(f"✅ График сохранён в Google Drive")
except:
    pass
print("✅ График сохранён: /content/training_history.png")

---
## 9. Финальная оценка на Test Set

In [ ]:
# Загружаем лучшую модель
model.load_state_dict(torch.load("/content/breed_best.pt"))
print("✅ Загружена лучшая модель")

# Оценка на test set
test_loss, test_top1, test_top5 = validate(
    model, test_loader, criterion, CONFIG["device"]
)

print("\n" + "="*60)
print("📊 РЕЗУЛЬТАТЫ НА TEST SET")
print("="*60)
print(f"Top-1 Accuracy: {test_top1*100:.2f}%")
print(f"Top-5 Accuracy: {test_top5*100:.2f}%")
print()

# Проверка target
if test_top5 > 0.80:
    print("✅ PASS: Top-5 > 80%")
else:
    print(f"❌ FAIL: Top-5 = {test_top5*100:.2f}% (target > 80%)")
    print("   Попробуй увеличить epochs или использовать более мощную модель")

---
## 10. Примеры предсказаний

In [ ]:
import random
from PIL import Image

# Выбираем случайные изображения
num_samples = 6
indices = random.sample(range(len(test_dataset)), num_samples)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

model.eval()
for i, idx in enumerate(indices):
    img_path, true_label = test_dataset.samples[idx]
    img = Image.open(img_path).convert("RGB")
    
    # Предсказание
    tensor = val_transform(img).unsqueeze(0).to(CONFIG["device"])
    with torch.no_grad():
        output = model(tensor)
        probs = torch.softmax(output, dim=1)[0]
        top5_probs, top5_idx = probs.topk(5)
    
    pred_label = top5_idx[0].item()
    pred_prob = top5_probs[0].item()
    
    true_name = id_to_breed[true_label]
    pred_name = id_to_breed[pred_label]
    
    # Отображение
    axes[i].imshow(img)
    color = "green" if pred_label == true_label else "red"
    axes[i].set_title(
        f"True: {true_name}\nPred: {pred_name} ({pred_prob*100:.1f}%)",
        color=color,
        fontsize=10,
    )
    axes[i].axis("off")

plt.tight_layout()
plt.savefig("/content/sample_predictions.png", dpi=150)
plt.show()

# Копируем в Google Drive (если подключён)
try:
    if os.path.exists("/content/drive/MyDrive"):
        shutil.copy("/content/sample_predictions.png", f"{PROJECT_DIR}/sample_predictions.png")
except:
    pass
print("✅ Примеры сохранены: /content/sample_predictions.png")

---
## 11. Сохранение результатов

In [ ]:
# Сохраняем конфигурацию и результаты
results = {
    "config": CONFIG,
    "best_epoch": best_epoch,
    "val_top5": best_top5,
    "test_top1": test_top1,
    "test_top5": test_top5,
    "history": history,
}

with open("/content/training_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Результаты сохранены!")

# Копируем в Google Drive (если подключён)
drive_connected = os.path.exists("/content/drive/MyDrive")
if drive_connected:
    try:
        shutil.copy("/content/training_results.json", f"{PROJECT_DIR}/training_results.json")
        print(f"\n📁 Файлы в Google Drive ({PROJECT_DIR}):")
    except:
        drive_connected = False

if not drive_connected:
    print(f"\n📁 Файлы в /content/:")

print("   - breed_best.pt (веса модели)")
print("   - breeds.json (маппинг пород)")
print("   - training_results.json (результаты)")
print("   - training_history.png (графики)")
print("   - sample_predictions.png (примеры)")

---
## 12. Скачивание модели

In [ ]:
# Скачай веса на локальный компьютер
from google.colab import files

print("Скачивание весов модели...")
files.download("/content/breed_best.pt")
files.download("/content/breeds.json")

print("\n✅ Файлы скачаны!")
print("\nСкопируй их в проект:")
print("  mv ~/Downloads/breed_best.pt models/breed.pt")
print("  mv ~/Downloads/breeds.json packages/models/breeds.json")

---
## ✅ Готово!

### Что дальше:

1. Скачай `breed_best.pt` на локальный компьютер
2. Скопируй в проект:
   ```bash
   mv ~/Downloads/breed_best.pt models/breed.pt
   ```
3. Протестируй:
   ```bash
   python scripts/demo/test_pipeline.py test/ShihTzu-original.jpeg
   ```

### Если Top-5 < 80%:
- Увеличь epochs до 50
- Попробуй `efficientnet_b5` или `vit_base_patch16_224`
- Добавь больше augmentation